In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from folktables import ACSDataSource, ACSPublicCoverage

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier

In [2]:
protected_attributes = [
    'SEX',
    'DIS',
    'NATIVITY',
    'DEAR',
    'DEYE',
    'MIG',
    'MIL',
    'AGEP',
    'DREM',
    'MAR',
]

In [8]:
data_source = ACSDataSource(survey_year='2018', horizon='5-Year', survey='person')

In [9]:
acs_data = data_source.get_data(join_household=True)

: 

In [7]:
len(acs_data)

KeyError: 0

In [3]:
features, label, group = ACSPublicCoverage.df_to_pandas(acs_data)

KeyError: 'AGEP'

In [6]:
# save the loaded data into a csv file
# features.to_csv('./my_data/features.csv', index=False)
# label.to_csv('./my_data/label.csv', index=False)
# group.to_csv('./my_data/group.csv', index=False)

In [3]:
# load the pandas data frames
features = pd.read_csv('./my_data/features.csv')
label = pd.read_csv('./my_data/label.csv')
group = pd.read_csv('./my_data/group.csv')

In [7]:
# load as numpy
features_np, label_np, group_np = ACSPublicCoverage.df_to_numpy(acs_data)

In [8]:
# save the numpy files
np.save('./my_data/features.npy', features_np)
np.save('./my_data/label.npy', label_np)
np.save('./my_data/group.npy', group_np)

In [18]:
for protect_attr in protected_attributes[:5]:
    print(f'Protected attribute: {protect_attr}')
    # Calculate prob_attr = P(protect_attr = 1) when attr can take 1 or 2
    prob_attr_1 = np.mean(features[protect_attr] == 1)
    prob_attr_0 = np.mean(features[protect_attr] == 2)
    print(f'{protect_attr} = 1: {prob_attr_1:.2f} and {protect_attr} = 2: {prob_attr_0:.2f}')

Protected attribute: SEX
SEX = 1: 0.43 and SEX = 2: 0.57
Protected attribute: DIS
DIS = 1: 0.15 and DIS = 2: 0.85
Protected attribute: NATIVITY
NATIVITY = 1: 0.85 and NATIVITY = 2: 0.15
Protected attribute: DEAR
DEAR = 1: 0.02 and DEAR = 2: 0.98
Protected attribute: DEYE
DEYE = 1: 0.03 and DEYE = 2: 0.97


In [27]:
# Calculate prob_attr = P(protect_attr = 1 or 4) when attr can take 0, 1, 2, 3, 4
protect_attr = 'MIL'
prob_attr_1 = np.mean((features[protect_attr] == 1) | (features[protect_attr] == 4))
prob_attr_0 = np.mean((features[protect_attr] == 0) | (features[protect_attr] == 2) | (features[protect_attr] == 3))
print(f'{protect_attr} = 1 or 4: {prob_attr_1:.2f} and {protect_attr} = 0, 2 or 3: {prob_attr_0:.2f}')

MIL = 1 or 4: 0.90 and MIL = 0, 2 or 3: 0.10


In [33]:
# Calculate prob_attr = P(protect_attr = 1) when attr can take 0, 1, 2, 3
protect_attr = 'MIG'
prob_attr_1 = np.mean(features[protect_attr] == 1)
prob_attr_0 = np.mean(features[protect_attr].isin([0, 2, 3]))
print(f'{protect_attr} = 1: {prob_attr_1:.2f} and {protect_attr} = 0, 2 or 3: {prob_attr_0:.2f}')

MIG = 1: 0.82 and MIG = 0, 2 or 3: 0.18


In [29]:
# Calculate prob_attr = P(protect_attr = 1) by thresholding at age 25
protect_attr = 'AGEP'
prob_attr_1 = np.mean(features[protect_attr] > 25)
prob_attr_0 = np.mean(features[protect_attr] <= 25)
print(f'{protect_attr} > 25: {prob_attr_1:.2f} and {protect_attr} <= 25: {prob_attr_0:.2f}')

AGEP > 25: 0.66 and AGEP <= 25: 0.34


In [32]:
# Calculate prob_attr = P(protect_attr = 1) when attr can take 1, 2, 3, 4, 5
protect_attr = 'MAR'
prob_attr_1 = np.mean(features[protect_attr] == 1)
prob_attr_0 = np.mean(features[protect_attr].isin([2, 3, 4, 5]))
print(f'{protect_attr} = 1: {prob_attr_1:.2f} and {protect_attr} = 0, 2, 3 or 4: {prob_attr_0:.2f}')

MAR = 1: 0.37 and MAR = 0, 2, 3 or 4: 0.63


In [35]:
# Calculate prob_attr = P(protect_attr = 1) when attr can take 0, 1, 2
protect_attr = 'DREM'
prob_attr_1 = np.mean(features[protect_attr] == 1)
prob_attr_0 = np.mean(features[protect_attr].isin([0,2]))
print(f'{protect_attr} = 1: {prob_attr_1:.2f} and {protect_attr} = 0, 2: {prob_attr_0:.2f}')

DREM = 1: 0.08 and DREM = 0, 2: 0.92


In [5]:
model = make_pipeline(StandardScaler(), GradientBoostingClassifier(loss='exponential', n_estimators=5, max_depth=5))

In [6]:
X_train, X_test, y_train, y_test, group_train, group_test = train_test_split(
    features, label, group, test_size=0.2, random_state=0)

In [7]:
model.fit(X_train, y_train)

/home/dhasade/anaconda3/envs/audits/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:424: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('gradientboostingclassifier',
                 GradientBoostingClassifier(loss='exponential', max_depth=5,
                                            n_estimators=5))])

In [ ]:
def load_dataset():
    # load the pandas data frames
    features = pd.read_csv('./my_data/features.csv')
    label = pd.read_csv('./my_data/label.csv')
    return features, label

In [8]:
y_pred = model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.7127843605199977


In [60]:
def demographic_parity(samples, y, attribute):
    # Calculate demographic parity for 'attribute'

    binary_attributes = ['SEX','DIS','NATIVITY','DEAR','DEYE']

    n = len(samples)

    if attribute in binary_attributes:
        # if the auditor doesn't test all subpopulations, we set that the demographic parity is null
        if not (0 < y[samples[attribute] == 1].sum().item() < n) or not (0 < y[samples[attribute] == 2].sum().item() < n):
            return 0

        prob_y_given_attribute_1 = y[samples[attribute] == 1].mean().item()  # P(y=1|attribute=1)
        prob_y_given_attribute_0 = y[samples[attribute] == 2].mean().item()  # P(y=1|attribute=0)

    elif attribute == 'MIL':
        prob_y_given_attribute_1 = y[samples[attribute].isin([1,4])].mean().item()
        prob_y_given_attribute_0 = y[samples[attribute].isin([0,2,3])].mean().item()

    elif attribute == 'MIG':
        prob_y_given_attribute_1 = y[samples[attribute] == 1].mean().item()
        prob_y_given_attribute_0 = y[samples[attribute].isin([0,2,3])].mean().item()
    
    elif attribute == 'AGEP':
        prob_y_given_attribute_1 = y[samples[attribute] > 25].mean().item()
        prob_y_given_attribute_0 = y[samples[attribute] <= 25].mean().item()
    
    elif attribute == 'MAR':
        prob_y_given_attribute_1 = y[samples[attribute] == 1].mean().item()
        prob_y_given_attribute_0 = y[samples[attribute].isin([2,3,4,5])].mean().item()

    elif attribute == 'DREM':
        prob_y_given_attribute_1 = y[samples[attribute] == 1].mean().item()
        prob_y_given_attribute_0 = y[samples[attribute].isin([0,2])].mean().item()

    else:
        raise ValueError('Attribute not supported')

    demographic_parity_attribute = abs(prob_y_given_attribute_1 - prob_y_given_attribute_0)
    return demographic_parity_attribute

In [49]:
protected_attributes = [
    'SEX',
    'DIS',
    'NATIVITY',
    'DEAR',
    'DEYE'
]

In [42]:
for attr in protected_attributes:
    dp = demographic_parity(features, label, attr)
    print(f'Demographic parity for {attr}: {dp:.2f}')

Demographic parity for SEX: 0.03
Demographic parity for DIS: 0.41
Demographic parity for NATIVITY: 0.05
Demographic parity for DEAR: 0.27
Demographic parity for DEYE: 0.31


In [61]:
demographic_parity(features, label, 'DREM')

0.4449089426372358

In [62]:
y_pred = model.predict(X_train)

In [66]:
y_pred.sum()

0

In [70]:
1.0 - label.mean().item()

0.7127970367941534

## Code for unbiasing DP estimation

In [5]:
from folk_tables import load_dataset, protected_attributes, class_mappings

In [3]:
X, y = load_dataset()

In [7]:
# Transform X based on class_mappings
X_transformed = X.copy()
for attr in protected_attributes:
    if attr == 'AGEP':
        X_transformed[attr] = X_transformed[attr].apply(lambda x: 1 if x > 25 else 0)
    else:
        class_mapping = class_mappings(attr)
        C1 = class_mapping[1] # list of values that are mapped to 1
        C0 = class_mapping[0] # list of values that are mapped to 0
        X_transformed[attr] = X_transformed[attr].apply(lambda x: 1 if x in C1 else 0)

In [12]:
from itertools import combinations

In [42]:
all_probs = dict()
n = 10

for k in range(1, n+1):
    print(f'Working on k={k}')
    all_probs[k] = dict()

    agent_combinations_list = list(combinations(range(n), k))

    for agent_combination in agent_combinations_list:
        agent_comb_str = ''.join([str(elem) for elem in agent_combination])
        
        all_probs[k][agent_comb_str] = dict()

        total_strings = 2**(k)
        binary_strings = [format(i, f'0{k}b') for i in range(total_strings)]

        attrs = [protected_attributes[i] for i in agent_combination]
        for binary_string in binary_strings:
            
            pairs = [(attrs[i], int(binary_string[i])) for i in range(k)]

            # Restore X_transformed that satisfies the binary string
            X_temp = X_transformed
            for attr, val in pairs:
                X_temp = X_temp[X_temp[attr] == val]

            all_probs[k][agent_comb_str][binary_string] = len(X_temp) / len(X_transformed)


Working on k=1
Working on k=2
Working on k=3


KeyboardInterrupt: 

In [38]:
# print lengths
for k in range(1, n+1):
    # String with all zeros of length k
    all_zeros = ''.join(['0' for _ in range(k)])
    a_key = list(all_probs[k].keys())[0]
    print(f'k={k}: {len(all_probs[k])} {len(all_probs[k][a_key])}')
    
print(sum([len(all_probs[k]) for k in range(1, n+1)]))

k=1: 10 2
k=2: 45 4
k=3: 120 8
k=4: 210 16
k=5: 252 32
k=6: 210 64
k=7: 120 128
k=8: 45 256
k=9: 10 512
k=10: 1 1024
1023


dict_keys(['0', '1'])